In [0]:
# Load data - Explore the smaller October dataset
df_oct = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

In [0]:
df_oct.write \
    .format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/ecommerce/ecommerce_data/df_oct_delta_day_6")


In [0]:
df_oct.write.format("delta").saveAsTable("df_oct_table_day_6_table")

BRONZE Medallion Layer - Raw unfiltered data (Incremental ingestion - append only)

In [0]:
# Read the source Delta table
# Loads Delta table as Spark DataFrame
df_bronze_raw = spark.table("df_oct_table_day_6_table")

# Write to Bronze Delta location / table
df_bronze_raw.write \
    # Writes in Delta Lake format
    .format("delta") \     
    .mode("overwrite") \
    # Saves as a managed Delta table named bronze_oct_events
    .saveAsTable("bronze_oct_events")


Check null values in each column

In [0]:
from pyspark.sql.functions import col, sum

# Count null values per column
df_bronze = spark.table("bronze_oct_events")

null_counts = df_bronze.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df_bronze.columns
])

display(null_counts)


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,0,0,0,13515609,6113008,0,0,2


SILVER medallion layer - Incremental cleaning & validation

Focus on the basics - Just handle null values in 'brand' and 'category_code'

In [0]:
from pyspark.sql.functions import col

# ==========================
# 1) Read Bronze
# ==========================
df_bronze = spark.table("bronze_oct_events")

# ==========================
# 2) Clean & Validate Data
# ==========================
df_silver = df_bronze.filter(
    (col("event_time").isNotNull()) &
    (col("event_type").isNotNull()) &
    (col("user_id").isNotNull())
)

# Fill null text fields
df_silver = df_silver.na.fill({
    "brand": "Not_Available",
    "category_code": "Not_Available"
})

# ==========================
# 3) Drop Duplicates
# ==========================
# Remove rows with same user_session + event_time
df_silver = df_silver.dropDuplicates(["user_session", "event_time"])

# ==========================
# 4) Write Silver Table
# ==========================
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_oct_events")


GOLD medallion layer - Incremental aggregates

#### KPI: Category Performance — Engagement vs Revenue

Measure how each category_code performs in terms of:
1. Views
2. Carts
3. Purchases
4. Revenue
5. Cart-to-Purchase Ratio

The conversion rate tells you the percentage of user sessions that resulted in a purchase out of all sessions that had activity.

In [0]:
from pyspark.sql import functions as F

# Read Silver
silver = spark.table("silver_oct_events")

# Aggregate by category
category_perf = silver.groupBy("category_code") \
    .agg(
        F.countDistinct(F.when(F.col("event_type") == "view", F.col("user_session"))).alias("unique_views"),
        F.countDistinct(F.when(F.col("event_type") == "cart", F.col("user_session"))).alias("unique_carts"),
        F.countDistinct(F.when(F.col("event_type") == "purchase", F.col("user_session"))).alias("unique_purchases"),
        F.sum(F.when(F.col("event_type") == "purchase", F.col("price"))).alias("total_revenue")
    ) \
    .withColumn(
        "cart_to_purchase_ratio",
        F.when(F.col("unique_carts") > 0,
               F.round(F.col("unique_purchases") / F.col("unique_carts") * 100, 2)
        ).otherwise(0)
    )

# Save as Gold Delta table
category_perf.write.format("delta").mode("overwrite").saveAsTable("gold_category_performance")


1. unique_views - Number of distinct user sessions that viewed the category
2. unique_carts - Number of distinct sessions that added a product in this category to cart
3. unique_purchases - Number of distinct sessions that bought something in this category
4. total_revenue - Total revenue generated from purchases in this category
5. cart_to_purchase_ratio - Total revenue generated from purchases in this category

Preview Category Perf

In [0]:
%sql
SELECT * FROM gold_category_performance ORDER BY total_revenue DESC LIMIT 10;


category_code,unique_views,unique_carts,unique_purchases,total_revenue,cart_to_purchase_ratio
electronics.smartphone,3098075,334695,285252,1.570331730000013E8,85.23
Not_Available,3383161,70470,154556,2.2921716160000067E7,219.32
computers.notebook,245148,10462,13655,8978883.419999978,130.52
electronics.video.tv,280535,23022,19221,8422119.38000002,83.49
electronics.clocks,373993,13647,15998,4816883.369999984,117.23
appliances.kitchen.washer,217839,13621,14507,4658223.4600000065,106.5
appliances.kitchen.refrigerators,215327,8489,10156,3829793.8899999964,119.64
electronics.audio.headphone,372117,32979,27403,3538807.1699999887,83.09
appliances.environment.vacuum,186160,11182,11570,1716425.4100000004,103.47
electronics.tablet,94107,5733,4934,1610917.4099999995,86.06
